In [2]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np
from functions import *
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline


In [4]:
pd.set_option('display.float_format','{:.3f}'.format)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

df = pd.read_csv("../data/01_raw/data.csv")
print(df.shape)

(1460, 81)


In [54]:
get_corr_pairs(df, 10)

0.88248 ('GarageArea', 'GarageCars')
0.82567 ('GarageYrBlt', 'YearBuilt')
0.82549 ('GrLivArea', 'TotRmsAbvGrd')
0.81953 ('TotalBsmtSF', '1stFlrSF')
0.79098 ('OverallQual', 'SalePrice')
0.70862 ('SalePrice', 'GrLivArea')
0.68750 ('2ndFlrSF', 'GrLivArea')
0.67662 ('TotRmsAbvGrd', 'BedroomAbvGr')
0.64921 ('BsmtFullBath', 'BsmtFinSF1')
0.64228 ('YearRemodAdd', 'GarageYrBlt')


In [55]:
cat_cols = df.select_dtypes(include="O").columns
num_cols = df.select_dtypes(exclude="O").columns

In [56]:
df.loc[:,cat_cols] = df.loc[:,cat_cols].fillna("NaN", axis=1)
df.loc[:,["MasVnrArea"]] = df.loc[:,["MasVnrArea"]].fillna(0.0)
df["GarageYrBlt"] = np.where(df["GarageYrBlt"].isna(), df["YearBuilt"], df["GarageYrBlt"])
df["LotFrontage"] = np.where(df["LotFrontage"].isna(), df["YearBuilt"].mean(), df["LotFrontage"])

In [60]:
 y = df.SalePrice
 df_features = cat_cols
 X = df[df_features]

le = LabelEncoder()
X_2 = X.apply(le.fit_transform)
X_2.head()
 



,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,3,1,1,3,3,0,4,0,5,2,2,0,5,1,1,12,13,1,2,4,2,2,4,4,2,6,1,0,1,5,2,6,3,1,2,5,5,2,3,4,1,8,4
1,3,1,1,3,3,0,2,0,24,1,2,0,2,1,1,8,8,3,3,4,1,2,4,1,0,6,1,0,1,5,3,6,5,1,2,5,5,2,3,4,1,8,4
2,3,1,1,0,3,0,4,0,5,2,2,0,5,1,1,12,13,1,2,4,2,2,4,2,2,6,1,0,1,5,2,6,5,1,2,5,5,2,3,4,1,8,4
3,3,1,1,0,3,0,0,0,6,2,2,0,5,1,1,13,15,3,3,4,0,4,1,4,0,6,1,2,1,5,2,6,2,5,3,5,5,2,3,4,1,8,0
4,3,1,1,0,3,0,2,0,15,2,2,0,5,1,1,12,13,1,2,4,2,2,4,0,2,6,1,0,1,5,2,6,5,1,2,5,5,2,3,4,1,8,4


In [ ]:
#  X_train, X_test, y_train, y_test = train_test_split(
#      X, y, random_state=2
#  )



# # (StandardScaler(), num_cols),
# preprocessor = make_column_transformer(
#     (OneHotEncoder(), cat_cols),
#     remainder='passthrough'
# )

# model = make_pipeline(
#     preprocessor,
#     LinearRegression()
# )


# _ = model.fit(X_train, y_train)

# # Affichage des β1 pour chaque variable
# for idx, name in enumerate(X_train.columns):
#     print(f"β1 de {name} : {round(_.coef_[idx], 3)}")

# print(f"β0 (intercept_) : {round(_.intercept_, 3)}\n")

# y_train_pred = _.predict(X_train)
# y_test_pred = _.predict(X_test)

# get_r2_rmse([y_train, y_train_pred, y_test, y_test_pred])